# HAND INDEX procedure

Procedure calculates HAND index from given DEM.

INPUT:
- DEM (*.tif)
- Stream Network (*.shp, lines vector layer)
- Stream nodes (*.shp, points vector layer)

Requirements:
- install TauDEM tools and set PATH
- install GDAL and set PATH
- install MinGW (for parallel mode)

NOTE:
It is desired that DEM contains the entire reach.
- aread8 (TauDEM): 
`By default, the tool checks for edge contamination. This is defined as the possibility that a contributing area value may be underestimated due to grid cells outside of the domain not being counted. This occurs when drainage is inwards from the boundaries or areas with no data values for elevation. The algorithm recognizes this and reports "no data" for the contributing area. It is common to see streaks of "no data" values extending inwards from boundaries along flow paths that enter the domain at a boundary. This is the desired effect and indicates that contributing area for these grid cells is unknown due to it being dependent on terrain outside of the domain of data available. Edge contamination checking may be turned off in cases where you know this is not an issue or want to ignore these problems, if for example, the DEM has been clipped along a watershed outline.`

## 0. Initalize

- import required modules
- set paths to source files
- obtain propertis of input DEM

In [1]:
from os.path import join

class MyPath:
    """Creates path for various files"""
    def __init__(self, main_dir, file_name):
        self.main_dir = main_dir
        self.main_name = file_name
        self.path_wo_ext = join(main_dir, file_name)
    
    def tif(self, extension=None):
        if extension:
            new_path = self.path_wo_ext + extension + ".tif"
        else:
            new_path = self.path_wo_ext + ".tif"
        return new_path

In [2]:
import subprocess
import os
import rasterio
from osgeo import gdal
from pyproj.crs import CRS

In [3]:
# INPUT PATHS

# Input DEM
# dem_original = "c:\\Users\\ncoz\\ARRS_susa\\dmv_10m.tif"
# dem_original = "c:\\Users\\ncoz\\ARRS_susa\\raw_sample\\sample_DEM_454_84_tap.tif"
# dem_original = "c:\\Users\\ncoz\\ARRS_susa\\sample_dmv_529_144.tif"
dem_original = "c:\\Users\\ncoz\\hand_index\\dmv_10m_nd999.tif"

# Input SHP files
# streams_in = "c:\\Users\\ncoz\\ARRS_susa\\DRSV_HIDRO5_LIN_PV\\DRSV_HIDRO5_LIN_PV.shp"
# nodes_in = "c:\\Users\\ncoz\\ARRS_susa\\DRSV_HIDRO5_LIN_PV\\DRSV_HIDRO5_LIN_PV_vertices.shp"
# streams_in = "c:\\Users\\ncoz\\hand_index\\DRSV_HIDRO5_LIN_PV\DRSV_HIDRO5_LIN_PV.shp"
# nodes_in = "c:\\Users\\ncoz\\hand_index\\DRSV_HIDRO5_LIN_PV\DRSV_HIDRO5_LIN_PV_vertices.shp"
streams_in = "c:\\Users\\ncoz\\hand_index\\DRSV_HIDRO5_LIN_PV\DRSV_HIDRO5_LIN_PV_filter_tiptv_1_5_stanje_4.shp"
nodes_in = "c:\\Users\\ncoz\\hand_index\\DRSV_HIDRO5_LIN_PV\DRSV_HIDRO5_LIN_PV_filter_tiptv_1_5_stanje_4_vertices.shp"

In [4]:
# IMPORTANT PARAMETERS

# Main working directory
# work_dir = "C:\\Users\\ncoz\\ARRS_susa"
work_dir = "C:\\Users\\ncoz\\hand_index"

# Folder to be put in the main directory for saving results
case_dir = "slo_03"

# Main name for out files, e.g. dem_<out_nam>_ext.tif
out_nam = "slo_03"

# Use nodes
use_nodes = True

In [5]:
# Create output folder
out_dir = os.path.join(work_dir, case_dir)
os.makedirs(out_dir, exist_ok=True)

In [6]:
# Create path objects
dem = MyPath(out_dir, "dem_" + out_nam)
streams = MyPath(out_dir, "streams_" + out_nam)
nodes = MyPath(out_dir, "nodes_" + out_nam)

# TauDEM MPI settings to enable parallel computations
mpi_settings = ["mpiexec", "-n", "22"]

# Get extent and epsg code of dem
with rasterio.open(dem_original) as ds:
    dem_bb = ds.bounds
    dem_res = ds.res
    dem_meta = ds.profile

# Assign CRS from pyproj
slo_crs = CRS.from_epsg(3794)

print("Check paths:")
if os.path.exists(dem_original):
    print(f"DEM    : {dem_original}")
if os.path.exists(streams_in):
    print(f"Nodes  : {streams_in}")
if os.path.exists(nodes_in):
    print(f"Streams: {nodes_in}")
print("\nCheck DEM propertis:")
print(dem_bb)
print(dem_res)
print("\nOutputs will be stored to:")
print(out_dir)

Check paths:
DEM    : c:\Users\ncoz\hand_index\dmv_10m_nd999.tif
Nodes  : c:\Users\ncoz\hand_index\DRSV_HIDRO5_LIN_PV\DRSV_HIDRO5_LIN_PV_filter_tiptv_1_5_stanje_4.shp
Streams: c:\Users\ncoz\hand_index\DRSV_HIDRO5_LIN_PV\DRSV_HIDRO5_LIN_PV_filter_tiptv_1_5_stanje_4_vertices.shp

Check DEM propertis:
BoundingBox(left=374000.0, bottom=31000.0, right=624010.0, top=195000.0)
(10.0, 10.0)

Outputs will be stored to:
C:\Users\ncoz\hand_index\slo_03


## 1. Rasterize shape files

- streams file (line features)
- nodes file (point features)

In [59]:
# Rasterize streams to match DEM
subprocess.call(
    " ".join([
        "gdal_rasterize",
        "-burn", "1",
        "-co", "COMPRESS=LZW",
        "-init", "0",
        "-tap",
        "-ot", "Byte",
        "-te", " ".join([str(i) for i in list(dem_bb)]),
        "-tr", str(dem_res[0]) + " " + str(dem_res[1]),
        streams_in,
        streams.tif()
    ])
)

0

In [60]:
# Rasterize vertices to match DEM
subprocess.call(
    " ".join([
        "gdal_rasterize",
        "-burn", "1",
        "-co", "COMPRESS=LZW",
        "-init", "0",
        "-tap",
        "-ot", "Byte",
        "-te", " ".join([str(i) for i in list(dem_bb)]),
        "-tr", str(dem_res[0]) + " " + str(dem_res[1]),
        nodes_in,
        nodes.tif()
    ])
)

0

## 2. DEM conditioning

A hydrologically conditioned DEM has no pits so that a drainage path can be defined from each grid cell to the edge of the domain. Each grid cell is conditioned to drain to one of the adjacent cells.


Create conditioned DEM:
- Fixing topology (breach obstacles, pit removal)
- Determining flow direction
- Mask flow directions by stream
- Soft burn the corrected topolgy to DEM
- Remove pits

### Burn streams into DEM

Basically we force breach any potential obstacels in the path of streams. Grid manipulation using rasterio `zb = z-100 * stream`. Can use any big number instead of 100.

IN:
- original DEM
- rasterized streams `streams.tif`

OUT:
- DEM with burned-in streams `dem_burn.tif`

In [61]:
# Burn streams into dem

# Create output raster
output_file = dem.tif("_burn")

# Open rasters
src_a = rasterio.open(dem_original)
src_b = rasterio.open(streams.tif())

grid_a = src_a.read()
grid_b = src_b.read()

print(grid_a.shape)
print(grid_b.shape)

# Raster calculator
grid_out = (grid_a - (grid_b * 100))

# Prepare metadata & save
out_meta = src_a.profile.copy()
out_meta.update({"crs":slo_crs})
with rasterio.open(output_file, "w", **out_meta) as dest:
    dest.write(grid_out)

src_a.close()
src_b.close()

print("DONE!")

(1, 16400, 25001)
(1, 16400, 25001)
DONE!


### Fill pits

`TauDEM:
Output Pit Removed Elevation Grid. A grid of elevation values with pits removed so that flow is routed off of the domain. Pits are low elevation areas in digital elevation models (DEMs) that are completely surrounded by higher terrain. They are generally taken to be artifacts of the digitation process that interfere with the processing of flow across DEMs. So, they are removed by raising their elevation to the point where they just drain.`

Filling pits prevents localized depressions and fill in the burned-in streams from previous step. If "burn" step was skipped, fill pits on the source DEM.

IN:
- DEM with burned in rasterized strems: `dem_burn.tif`

OUT:
- Pit-filled DEM with burned in streams: `dem_burn_fel.tif`

In [62]:
# Fill pits
print(
    mpi_settings + [
    "pitremove", 
    "-z", dem.tif("_burn"),
    "-fel", dem.tif("_burn_fel")
    ]
)
subprocess.call(
    mpi_settings + [
        "pitremove", 
        "-z", dem.tif("_burn"),
        "-fel", dem.tif("_burn_fel")
    ]
)
with rasterio.open(dem.tif("_burn_fel"), "r+") as rio:
    rio.crs = slo_crs

['mpiexec', '-n', '22', 'pitremove', '-z', 'C:\\Users\\ncoz\\hand_index\\slo_02\\dem_slo_02_burn.tif', '-fel', 'C:\\Users\\ncoz\\hand_index\\slo_02\\dem_slo_02_burn_fel.tif']


### Determine flow directions

`TauDEM: Takes as input the hydrologically correct elevation grid and outputs D8 flow direction and slope for each grid cell. In flat areas flow directions are assigned away from higher ground and towards lower ground.`

Gives two rasters. In the first one, each pixel has value 1-8, representing the direction of the flow in that pixel. The other raster provides slope angles for each pixel.

IN:
- Pit-filled DEM with burned in streams: `dem_burn_fel.tif`

OUT:
- Flow directions: `dem_burn_fel_p.tif`
- Slope: `dem_burn_fel_sd8.tif` (UNUSED)

In [63]:
# Determine flow dirs
subprocess.call(
    mpi_settings + [
        "d8flowdir",
        "-fel", dem.tif("_burn_fel"),
        "-p", dem.tif("_burn_fel_p"),
        "-sd8", dem.tif("_burn_fel_sd8")
    ]
)
with rasterio.open(dem.tif("_burn_fel_p"), "r+") as rio:
    rio.crs = slo_crs
with rasterio.open(dem.tif("_burn_fel_sd8"), "r+") as rio:
    rio.crs = slo_crs

### Mask streams

Apply streams mask - we only need directions in the stream network itself. Use raserio for grid calculation.

IN:
- Flow directions `dem_burn_fel_p.tif`
- Rasterized stream network `streams.tif`

OUT:
- Stream network with flow directions `dem_burn_fel_p_masked.tif`

In [64]:
# Mask flow dirs by streams

# Create output raster
output_file = dem.tif("_burn_fel_p_masked")

# Open rasters
src_a = rasterio.open(dem.tif("_burn_fel_p"))
src_b = rasterio.open(streams.tif())

grid_a = src_a.read()
grid_b = src_b.read()

print(grid_a.shape)
print(grid_b.shape)

# Raster calculator
grid_out = (grid_a * grid_b)

# Prepare output metadata & save
out_meta = src_a.profile.copy()
out_meta.update({"crs":slo_crs, "dtype":'int16'})
with rasterio.open(output_file, "w", **out_meta) as dest:
    dest.write(grid_out)

src_a.close()
src_b.close()

print("DONE!")

(1, 16400, 25001)
(1, 16400, 25001)
DONE!


### DEM conditioning

Apply flow directions to the original DEM.

`TauDEM: Operates on -pm and -z and produces -zsb (soft burned elevations) by tracking down the D8 flow directions and ensuring there is no uphill elevation.`

IN:
- original_dem
- Stream network with flow directions `dem_burn_fel_p_masked.tif`

OUT:
- Conditioned DEM `dem_burn_fel_p_masked_zfcd.tif`

In [65]:
# Condition DEM
subprocess.call(
    mpi_settings + [
        "flowdircond",
        "-z", dem_original,
        "-p", dem.tif("_burn_fel_p_masked"),
        "-zfdc", dem.tif("_burn_zfdc")
    ]
)
with rasterio.open(dem.tif("_burn_zfdc"), "r+") as rio:
    rio.crs = slo_crs
    # TODO: ali ima to smisel da spremenim nodata v -9999?
    rio.nodata = -999

### Remove pits on conditioned DEM

Finally, pit removal is performed on the conditioned DEM, obtaining the end product of this sub-routine `dem_cond.tif`.

The conditioned DEM with rmoved pits will be used as input for the next processing sub-routines (3. STREAM NETWORK) and (4. HAND INDEX).

`TauDEM: Output Pit Removed Elevation Grid. A grid of elevation values with pits removed so that flow is routed off of the domain. Pits are low elevation areas in digital elevation models (DEMs) that are completely surrounded by higher terrain. They are generally taken to be artifacts of the digitation process that interfere with the processing of flow across DEMs. So, they are removed by raising their elevation to the point where they just drain.`

IN:

- Conditioned DEM `dem_burn_fel_p_masked_zfdc.tif`

OUT:

- Pit-filled conditioned DEM `dem_cond.tif`


In [66]:
# Remove pits on conditioned DEM
subprocess.call(
    mpi_settings + [
        "pitremove", 
        "-z", dem.tif("_burn_zfdc"),
        "-fel", dem.tif("_cond")
    ]
)
with rasterio.open(dem.tif("_cond"), "r+") as rio:
    rio.crs = slo_crs

## 3. Stream network raster

Using the conditioned elevation model with removed pits, we proceed with the (3.1) calculation of flow directions and (3.2) delineation of the stream network using the start/end points as weights.
This stream network is similar to the one obtained from the vector layer, and is now consistent with flow directions obtained from the DEM.

### Determine flow directions on conditioned DEM

`TauDEM: Takes as input the hydrologically correct elevation grid and outputs D8 flow direction and slope for each grid cell. In flat areas flow directions are assigned away from higher ground and towards lower ground.`

IN:
- Conditioned DEM `dem_cond.tif`

OUT:
- Flow directions: `dem_cond_p.tif`
- Slope: `dem_cond_sd8.tif` (UNUSED)


In [67]:
# Determine flow dirs on conditioned dem
subprocess.call(
    mpi_settings + [
        "d8flowdir",
        "-fel", dem.tif("_cond"),
        "-p", dem.tif("_cond_p"),
        "-sd8", dem.tif("_cond_sd8")
    ]
)
with rasterio.open(dem.tif("_cond_p"), "r+") as rio:
    rio.crs = slo_crs
with rasterio.open(dem.tif("_cond_sd8"), "r+") as rio:
    rio.crs = slo_crs

### Flow accumulation

`TauDEM: Takes as input a D8 flow directions grid and outputs the contributing area as the number of grid cells draining through each grid cell (Optionally accumulates an input weight grid).`

The value of the pixel tells you how many pixels there are upstream/upslope of the selected pixel.

IN:
- Flow directions `dem_cond_p.tif`

OUT:
- Contributing area `dem_cond_p_ad8.tif`

In [68]:
# Flow accumulation on conditioned dem (LOOKS LIKE IT WAS UNUSED!)
if not use_nodes:
    subprocess.call(
        mpi_settings + [
            "aread8", 
            "-p", dem.tif("_cond_p"),
            "-ad8", dem.tif("_cond_p_ad8"),
            "-nc"
        ]
    )
    with rasterio.open(dem.tif("_cond_p_ad8"), "r+") as rio:
        rio.crs = slo_crs

### Flow accumulation weighted by stream nodes

`TauDEM: Takes as input a D8 flow directions grid and outputs the contributing area as the number of grid cells draining through each grid cell (Optionally accumulates an input weight grid).`

Here, the aread8 is used to accumulate an input weight grid.

(It is the only place I am using strem nodes extracted from stream vector file.)

IN:
- Flow directions `dem_cond_p.tif`
- Rasteized nodes `nodes.tif`

OUT:
- Flow accumulation weighted by stream nodes `dem_cond_p_ssa.tif`

In [7]:
# Flow accumulation weighted by stream start and end points
if use_nodes:
    subprocess.call(
        mpi_settings + [
            "aread8", 
            "-p", dem.tif("_cond_p"),
            "-ad8", dem.tif("_cond_p_ssa"),
            "-wg", nodes.tif(),
            "-nc"
        ]
    )
    with rasterio.open(dem.tif("_cond_p_ssa"), "r+") as rio:
        rio.crs = slo_crs

### Delineate streams by threshold

`TauDEM: Takes as input any grid and outputs an indicator (1,0) grid of grid cells that have values >= the input threshold. This is used to delineate stream networks from contributing area and similar grids.`

In literature threshold 50 is suggested, however this is much to low. In literature 3" SRTM data is used, which gives pixel size 90m at equator. We are using downsampled ALS DEM with pixel size 10m. Therfore 50 is much to low and gives a very dense stream network, with too many non-existent small tributarys, which lower the overall value of HAND index. This is especially pronunced in Kras (casrst) region, where poprous land results in almost no streams on the surface.

The Danish guy used threshold 1, but stream network was weighted with stream nodes extracted from shp file.

IN:
- Flow accumulation weighted by stream nodes `dem_cond_p_ssa.tif`

OUT:
- Stream network for HAND calculation `dem_src.tif`

In [8]:
# Delineate streams by threshold (OPTIONB)
if use_nodes:
    ad8_file = dem.tif("_cond_p_ssa")
    thr = "1"
else:
    ad8_file = dem.tif("_cond_p_ad8")
    thr = "500"
subprocess.call(
    mpi_settings + [
        "threshold", 
        "-ssa", ad8_file,
        "-src", dem.tif("_src"),
        "-thresh", thr
    ]
)
with rasterio.open(dem.tif("_src"), "r+") as rio:
    rio.crs = slo_crs

## 4. Calculate HAND

Finally, we calculate D-Inf flow directions and HAND. The result is a grid where each cell denotes the vertical height above the nearest stream, with “nearest” being the nearest stream cell in along the D-Inf flowpath. In the final step, we use some raster math to assign zero to the coastal area for which HAND has not been calculated.

### Infinity flow directions

`TauDEM: Assigns a flow direction based on steepest slope on a triangular facet following the D∞ model. This is recorded as an angle in radians anticlockwise from east.`

IN:
- Conditioned DEM `dem_cond.tif`

OUT:
- Slope grid `dem_cond_slp.tif`
- Flow direction grid `dem_cond_ang.tif`

In [9]:
# Calculate infinity flow directions
subprocess.call(
    mpi_settings + [
        "dinfflowdir",
        "-fel", dem.tif("_cond"),
        "-slp", dem.tif("_cond_slp"),
        "-ang", dem.tif("_cond_ang")
    ]
)
with rasterio.open(dem.tif("_cond_slp"), "r+") as rio:
    rio.crs = slo_crs
with rasterio.open(dem.tif("_cond_ang"), "r+") as rio:
    rio.crs = slo_crs

### Calculate HAND raster

`TauDEM: Calculates distance downslope to a target zone (typically stream)
using Dinf flow directions. Options include vertical, horizontal, along
slope and pythagorus distances, computed using minimum,
maximum, or flow weighted averaging along multiple Dinf flow paths.`

IN:
- Conditioned DEM `dem_cond.tif`
- Slope grid `dem_cond_slp.tif`
- Flow direction grid `dem_cond_ang.tif`
- Stream network for HAND calculation `dem_src.tif`

OUT:
- HAND index `dem_HAND.tif`

In [10]:
# Calculate HAND raster
subprocess.call(
    mpi_settings + [
        "dinfdistdown",
        "-fel", dem.tif("_cond"),
        "-slp", dem.tif("_cond_slp"),
        "-ang", dem.tif("_cond_ang"),
        "-src", dem.tif("_src"),
        "-dd", dem.tif("_HAND"),
        "-m",  "v",  "ave"
    ]
)
with rasterio.open(dem.tif("_HAND"), "r+") as rio:
    rio.crs = slo_crs

### Some post-processig to make it look nicer

Assign zero to areas for which HAND could not be calculated (e.g. coastal areas) and assign nodata values based on the original DEM.

IN:
- HAND index: `dem_HAND.tif`
- original DEM

OUT:
- HAND index (final version) `dem_HAND_final.tif`

In [11]:
# Fill HAND raster with zero on land surfaces where HAND is nodata
with rasterio.open(dem.tif("_HAND")) as grid:
    handgrid = grid.read()
    handmeta = grid.profile

handgrid[handgrid < 0] = 0

# This one is used to set all nodata from original dem to -1
with rasterio.open(dem_original) as grid:
    demgrid = grid.read()

# For SLovenian DEM, nodata in original raster is 0
handgrid[demgrid == -999] = -1

# Update metadata
handmeta["nodata"] = -1
handmeta.update({"crs":slo_crs})

# Save
with rasterio.open(dem.tif("_HAND_final"), "w", **handmeta) as dst:
    dst.write(handgrid)

print("DONE!")

DONE!


At the end, there was some interpolation done for the rest of the missing data. Some polygons were also located over the border, so the raster was interpolated 600 pixels out of the border. The NODATA of the final product is set to very big negative nnumber. Change it to np.nan!

In [1]:
import numpy as np
import rasterio

fill_rst = "c:\\Users\\ncoz\\ARRS_susa\\fill_tif_fill\\dem_slo_03_HAND_fill.tif"
fill_out = "c:\\Users\\ncoz\\ARRS_susa\\fill_tif_fill\\dem_slo_03_HAND_fill_nan.tif"

with rasterio.open(fill_rst) as grid:
    handgrid = grid.read()
    handmeta = grid.profile



In [2]:
handmeta

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028234663852886e+38, 'width': 25001, 'height': 16400, 'count': 1, 'crs': CRS.from_epsg(3794), 'transform': Affine(10.0, 0.0, 374000.0,
       0.0, -10.0, 195000.0), 'tiled': False, 'interleave': 'band'}

In [3]:
handgrid[handgrid < 0] = np.nan
handmeta.update({"nodata":np.nan})

# Save
with rasterio.open(fill_out, "w", **handmeta) as dst:
    dst.write(handgrid)